# Modify CAM output


<div class="alert alert-info">
<strong>Exercise: Customize your CAM history files</strong><br><br>
 
Create a case called **b1850_high_freq** using the compset  ``B1850``  at ``f19_g17`` resolution. 
    
Set the run length to **1 month**. 

In addition to the monthly history file ``h0``, output:
- ``h1`` file with **instantaneous** values of T, Q, U and V every **24 hour**.
- ``h2`` file with **time-average** values of T, Q, U and V every **3 hour**.

Set your namelist so that you output:  
- a **single** ``h1`` file with all the daily output for the month. 
- **multiple** ``h2`` files, one for every day of the month.

Your goal is to produce:
- one ``h1`` file with 31 timesteps and 
- thirty-one ``h2`` files with 8 timesteps each.

Make a 1-month run.

</div>



<div class="alert alert-warning">  
<details>

<summary> <font face="Times New Roman" color='blue'>Click here for hints</font> </summary>
    
- **How do I compile?**

You can compile with the command:
```
qcmd -- ./case.build
```

- **How do I control the output?**

Use namelist variables: ``nhtfrq``, ``mfilt``, ``fincl``. 

Look at the online documentation for these variables.

- **How do I check my solution?**

When your run is completed, go to the archive directory. 

(1) Check that your archive directory contains the files:

- ``h0`` files
```
b1850_high_freq.cam.h0.0001-01.nc
```
- ``h1`` files
```
b1850_high_freq.cam.h1.0001-01-01-00000.nc
b1850_high_freq.cam.h1.0001-02-01-00000.nc
```
- ``h2`` files
```
b1850_high_freq.cam.h2.0001-01-01-00000.nc
…
b1850_high_freq.cam.h2.0001-01-31-00000.nc
b1850_high_freq.cam.h2.0001-02-01-00000.nc
```

(2) Compare the contents of the ``h1`` and ``h2`` files using ``ncdump``.

```
ncdump -h b1850_high_freq.cam.h1.0001-01-01-00000.nc
ncdump -h b1850_high_freq.cam.h2.0001-01-01-00000.nc
```

(3) Check the number of timesteps in the ``h1`` and the ``h2`` files.
Look at the sizes of the files. 

</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
                
Create a new case <font face="Courier" color='purple'><strong>b1850_high_freq </strong></font> with the command:
```
cd /glade/work/$USER/code/my_cesm_code/cime/scripts/
./create_newcase --case ~/cases/b1850_high_freq  --compset B1850 --res f19_g17 
```

Case <font face="Courier" color='purple'><strong>setup</strong></font>:
```    
cd ~/cases/b1850_high_freq 
./case.setup
```
  
Edit the file <font face="Courier" color='purple'><strong>user_nl_cam</strong></font> and add the lines:
```
 nhtfrq = 0, -24, -3
 mfilt = 1, 31, 8 
 fincl2 = 'T:I','Q:I','U:I','V:I'
 fincl3 = 'T','Q','U','V'
```
    
Change the <font face="Courier" color='purple'><strong>run length</strong></font>:
```   
./xmlchange STOP_N=1,STOP_OPTION=nmonths
```

If needed, change <font face="Courier" color='purple'><strong>job queue</strong></font> 
and <font face="Courier" color='purple'><strong>account number</strong></font>. 
For instance:
```  
./xmlchange JOB_QUEUE=regular,PROJECT=UESM0011
```

Build and submit:
```
qcmd -- ./case.build
./case.submit
```

When the run is completed, look into the archive directory for: 
<font face="Courier" color='purple'><strong>b1850_high_freq</strong></font>.  
    
(1) Check that your archive directory on cheyenne (The path will be different on otehr machines): 
```
cd /glade/scratch/$user/archive/b1850_high_freq/atm/hist
ls 
```

(2) Compare the contents of the ``h1`` and ``h2`` files using ``ncdump``.
Look at the variables attributes. What is the difference between the 2 files ? 

The file with the instantaneous output ``h1`` have no cell_methods attribute while the average output ``h2`` has a attribute: 
```
cell_methods = "time: mean"
```

For instance for the field Q.

In the instantaneous file ``b1850_high_freq.cam.h1.0001-01-01-00000.nc``
```
float Q(time, lev, lat, lon) ;
                Q:mdims = 1 ;
                Q:units = "kg/kg" ;
                Q:mixing_ratio = "wet" ;
                Q:long_name = "Specific humidity" ;
```
In the time-averaged file  ``b1850_high_freq.cam.h2.0001-01-01-00000.nc``
```
float Q(time, lev, lat, lon) ;
                Q:mdims = 1 ;
                Q:units = "kg/kg" ;
                Q:mixing_ratio = "wet" ;
                Q:long_name = "Specific humidity" ;
                Q:cell_methods = "time: mean" ;
```
(3) Check the number of timesteps in the h1 and the h2 files. 

- ``h1`` contains 31 time samples.  
In the netcdf file,  
```
time = UNLIMITED ; // (31 currently)
```
- ``h2`` contains 8 time samples 
In the netcdf file, 
```
time = UNLIMITED ; // (8 currently)
```
- Check the size of the files
```
du –ks –h /glade/scratch/$user/archive/b1850_high_freq/atm/hist/*
```
```
234M    b1850_high_freq.cam.h0.0001-01.nc

210M    b1850_high_freq.cam.h1.0001-01-01-00000.nc
7.0M    b1850_high_freq.cam.h1.0001-02-01-00000.nc

55M     b1850_high_freq.cam.h2.0001-01-01-00000.nc
55M     b1850_high_freq.cam.h2.0001-01-02-00000.nc
...
55M     b1850_high_freq.cam.h2.0001-01-31-00000.nc
7.0M    b1850_high_freq.cam.h2.0001-02-01-00000.nc
```

</details>
</div>

